In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
import pandas as pd
import numpy as np
import seaborn as sns

In [55]:
df = pd.read_csv('/Users/kasper/OneDrive/study/UvA/Master/Data mining/Assignments/Assignment 1 - advanced/dataset_mood_smartphone.csv')
df.head()

,Unnamed: 0,id,time,variable,value
0,1,AS14.01,2014-02-26 13:00:00.000,mood,6.0
1,2,AS14.01,2014-02-26 15:00:00.000,mood,6.0
2,3,AS14.01,2014-02-26 18:00:00.000,mood,6.0
3,4,AS14.01,2014-02-26 21:00:00.000,mood,7.0
4,5,AS14.01,2014-02-27 09:00:00.000,mood,6.0


In [56]:
df['time'] = pd.to_datetime(df['time'])

In [57]:
df2 = df.set_index('time').groupby(['id', 'variable']).resample('D').mean().reset_index()
df2 = df2.drop('Unnamed: 0',axis=1)
df2.head()

,id,variable,time,value
0,AS14.01,activity,2014-03-20,0.081548
1,AS14.01,activity,2014-03-21,0.134050
2,AS14.01,activity,2014-03-22,0.236880
3,AS14.01,activity,2014-03-23,0.142741
4,AS14.01,activity,2014-03-24,0.078961


In [63]:
pivoted = df2.pivot(index=['id', 'time'], columns='variable', values='value').reset_index()
pivoted.columns.name=None
pivoted = pivoted[pivoted['mood'].notna()]
pivoted.head(20)

,id,time,activity,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,...,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,circumplex.arousal,circumplex.valence,mood,screen,sms
9,AS14.01,2014-02-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,-0.250000,0.750000,6.250000,NaN,1.0
10,AS14.01,2014-02-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.333333,6.333333,NaN,NaN
32,AS14.01,2014-03-21,0.134050,14.269173,55.095526,53.024000,6.193000,NaN,57.40200,29.968875,...,70.418846,NaN,23.029000,NaN,1.0,0.200000,0.200000,6.200000,153.665872,NaN
33,AS14.01,2014-03-22,0.236880,7.781160,51.697063,46.662000,7.025333,NaN,NaN,14.020429,...,37.305000,NaN,39.207000,NaN,1.0,0.600000,0.500000,6.400000,47.613651,1.0
34,AS14.01,2014-03-23,0.142741,12.134396,53.442031,31.448667,10.850750,NaN,NaN,12.137167,...,NaN,NaN,30.086000,30.386,NaN,0.200000,0.800000,6.800000,67.067644,NaN
35,AS14.01,2014-03-24,0.078961,4.982506,47.541687,122.121375,5.684333,NaN,3.01000,11.093000,...,83.961000,NaN,89.366000,NaN,1.0,0.800000,0.000000,6.000000,76.772199,NaN
36,AS14.01,2014-03-25,0.098374,10.026874,69.437507,34.103000,10.763500,NaN,NaN,29.803167,...,NaN,29.402875,22.289300,NaN,NaN,0.500000,0.500000,6.750000,169.097283,1.0
37,AS14.01,2014-03-26,0.101308,8.460123,47.061534,130.068429,8.721833,NaN,NaN,16.249667,...,NaN,NaN,33.365000,NaN,NaN,-0.200000,0.600000,6.600000,94.389661,NaN
38,AS14.01,2014-03-27,0.159511,7.982643,43.353200,71.037667,14.073000,NaN,15.20425,9.755333,...,23.657000,NaN,16.275364,NaN,1.0,0.200000,0.800000,7.000000,129.201111,1.0
39,AS14.01,2014-03-28,0.095698,81.149960,24.078694,91.400273,12.738000,116.518,NaN,37.658500,...,59.632053,NaN,30.171700,NaN,1.0,-0.600000,0.600000,6.400000,55.948739,NaN


In [59]:
#pivoted.to_csv('pivoted_v1.csv')